In [99]:
import requests
from bs4 import BeautifulSoup

import pandas as pd
from pandas import DataFrame

page_basic = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Basic1.aspx"
page_detail = "https://www.koreabaseball.com/Record/Player/PitcherBasic/Detail1.aspx"
res = requests.get(page_basic)

soup = BeautifulSoup(res.text, 'lxml')
ths = soup.find_all('th')
tds = soup.find_all('td')

head = []
for th in ths:
    head.append(th.get_text())

data = []
for i, td in enumerate(tds):
    try:
        data.append(int(td.get_text()))
    except:
        try:
            data.append(float(td.get_text()))
        except:
            data.append(td.get_text())

parent_arr = []
base = len(head)
row = int(len(data) / base);
for i in range(base):
    node_arr = [];
    for j in range(row):
        offset = i + (base * j);
        node_arr.append(data[offset]);
    parent_arr.append(node_arr);

a = pd.DataFrame(parent_arr)
a = a.T
a.columns = head

res = requests.get(page_detail)

soup = BeautifulSoup(res.text, 'lxml')
ths = soup.find_all('th')
tds = soup.find_all('td')

head = []
for th in ths:
    head.append(th.get_text())

data = []
for i, td in enumerate(tds):
    try:
        data.append(int(td.get_text()))
    except:
        try:
            data.append(float(td.get_text()))
        except:
            data.append(td.get_text())

parent_arr = []
base = len(head)
row = int(len(data) / base);
for i in range(base):
    node_arr = [];
    for j in range(row):
        offset = i + (base * j);
        node_arr.append(data[offset]);
    parent_arr.append(node_arr);

b = pd.DataFrame(parent_arr)
b = b.T
b.columns = head

c = pd.merge(a, b)
c

,순위,선수명,팀명,ERA,G,W,L,SV,HLD,WPCT,...,GS,Wgs,Wgr,GF,SVO,TS,GDP,GO,AO,GO/AO
0,1,김광현,SSG,1.82,19,10,2,0,0,0.833,...,19,10,0,0,0,0,10,123,117,1.05
1,2,안우진,키움,2.17,21,11,5,0,0,0.688,...,21,11,0,0,0,0,6,117,135,0.87
2,3,요키시,키움,2.31,21,8,5,0,0,0.615,...,21,8,0,0,0,0,11,176,88,2.0
3,4,폰트,SSG,2.33,22,13,5,0,0,0.722,...,22,13,0,1,0,0,5,129,167,0.77
4,5,수아레즈,삼성,2.41,21,4,6,0,0,0.4,...,20,4,0,0,0,0,10,126,101,1.25
5,6,루친스키,NC,2.48,22,8,8,0,0,0.5,...,22,8,0,0,0,0,8,144,117,1.23
6,7,플럿코,LG,2.56,21,11,4,0,0,0.733,...,21,11,0,0,0,0,8,83,172,0.48
7,8,소형준,KT,2.76,19,11,2,0,0,0.846,...,19,11,0,0,0,0,13,169,98,1.72
8,9,켈리,LG,2.88,19,12,2,0,0,0.857,...,19,12,0,0,0,0,13,126,105,1.2
9,10,반즈,롯데,2.94,24,10,9,0,0,0.526,...,24,10,0,0,0,0,12,168,143,1.17
